In [ ]:
from google.colab import drive
import os
# Mount Google Drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/NCS
path = os.getcwd()
print(path)

Mounted at /content/gdrive
/content/gdrive/MyDrive/NCS
/content/gdrive/MyDrive/NCS


In [ ]:
import gzip
import pandas as pd

file_path ='Gene Data/'data_name'/data.trn.gz'

with gzip.open(file_path, 'rt') as f:
   df = pd.read_csv(f, header=None, sep='\s+');
   print(df.shape)
X = df.iloc[:, :-1]
y= df.iloc[:,-1]
unique_labels = y.unique()

num_classes = y.nunique()
print("Số lượng nhãn khác nhau:", num_classes)
label_counts = y.value_counts()
print("Số lượng mẫu cho mỗi nhãn:")
print(label_counts)
if 13 in label_counts.index:
    print(f"Số lượng mẫu cho nhãn 13: {label_counts[13]}")
else:
    print("Nhãn 13 không có trong tập dữ liệu.")
missing_labels = y.isnull().sum()
print(f"Số lượng mẫu không có nhãn (NaN): {missing_labels}")

In [ ]:
!pip install xgboost
!pip install boruta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
##  BORUTA Chạy trên các daataset - Ghi file kết quả sumary

import os
import logging
import warnings
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

datasets = [20685, 20711, 21050, 21122, 29354, 30784, 31312, 31552, 32537, 33315, 36895, 37364, 39582, 39716, 44077]


def load_data(file_path):
    try:
        df = pd.read_csv(file_path, header=None, sep='\s+')
        logging.info(f"Đọc dữ liệu từ file: {file_path}")
        return df
    except FileNotFoundError:
        logging.error(f"Không tìm thấy file: {file_path}")
        return None


def scale_data(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    logging.info("Dữ liệu đã được chuẩn hóa.")
    return X_scaled

def select_features_with_boruta(X_scaled, y):
    rf = RandomForestClassifier(n_estimators=200, random_state=42)
    boruta_selector = BorutaPy(
        rf,
        n_estimators=300,
        max_iter=200,
        alpha=0.02,
        perc=100,
        two_step=True,
        random_state=42,
        verbose=1
    )
    start_time = time.time()
    boruta_selector.fit(X_scaled, y)
    end_time = time.time()

    feature_selection_time = end_time - start_time
    logging.info(f"Thời gian trích đặc trưng bằng Boruta: {feature_selection_time:.2f} giây")
    print(f"Thời gian trích đặc trưng bằng Boruta: {feature_selection_time:.2f} giây")

    X_selected = X_scaled[:, boruta_selector.support_]
    num_selected_features = boruta_selector.support_.sum()
    logging.info(f"Số đặc trưng được chọn: {num_selected_features}")
    return X_selected, boruta_selector, num_selected_features, feature_selection_time
def save_selected_feature_names_to_csv(X, selected_features, output_directory, filename='selected_feature_names.csv'):
    selected_feature_names = X.columns[selected_features]
    selected_features_file = os.path.join(output_directory, filename)
    pd.DataFrame(selected_feature_names, columns=['Selected Features']).to_csv(selected_features_file, index=False)
    logging.info(f"Tên các đặc trưng đã được chọn lưu vào file: {selected_features_file}")
    print(f"Tên các đặc trưng đã được chọn lưu vào file: {selected_features_file}")

# Lưu kết quả Boruta vào file CSV
def save_boruta_summary_to_csv(boruta_selector, feature_selection_time, output_directory, filename='boruta_summary.csv'):

    confirmed_count = boruta_selector.support_.sum()
    tentative_count = boruta_selector.support_weak_.sum()
    rejected_count = (~(boruta_selector.support_ | boruta_selector.support_weak_)).sum()

    summary_df = pd.DataFrame({
        'Confirmed': [confirmed_count],
        'Tentative': [tentative_count],
        'Rejected': [rejected_count],
        'Feature Selection Time (s)': [feature_selection_time]
    })


    summary_file = os.path.join(output_directory, filename)
    summary_df.to_csv(summary_file, index=False)
    logging.info(f"Tổng kết Boruta được lưu vào file: {summary_file}")
    print(f"Tổng kết Boruta được lưu vào file: {summary_file}")

all_results = pd.DataFrame(columns=['Dataset', 'Confirmed', 'Tentative', 'Rejected', 'Feature Selection Time (s)'])
for dataset in datasets:

    file_path = f'Gene Data/{dataset}/data.trn.gz'
    output_directory = os.path.join('Thuc Nghiem', 'Selected Features', 'Results', '09.10', str(dataset))
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    log_file = os.path.join(output_directory, 'experiment_log.log')
    logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info(f"Bắt đầu quá trình chọn lọc đặc trưng với Boruta cho dataset: {dataset}.")
    df = load_data(file_path)
    if df is not None:
        X = df.iloc[:, :-1]
        y = df.iloc[:, -1]
        X_scaled = scale_data(X)
        X_selected, boruta_selector, num_selected_features, feature_selection_time = select_features_with_boruta(X_scaled, y)
        print(f"Số đặc trưng được chọn cho dataset {dataset}: {num_selected_features}")
        save_selected_feature_names_to_csv(X, boruta_selector.support_, output_directory)
        save_boruta_summary_to_csv(boruta_selector, feature_selection_time, output_directory)
        confirmed_count = boruta_selector.support_.sum()
        tentative_count = boruta_selector.support_weak_.sum()
        rejected_count = (~(boruta_selector.support_ | boruta_selector.support_weak_)).sum()
        all_results = pd.concat([all_results, pd.DataFrame([{
            'Dataset': dataset,
            'Confirmed': confirmed_count,
            'Tentative': tentative_count,
            'Rejected': rejected_count
            'Feature Selection Time (s)': feature_selection_time
        }])], ignore_index=True)

output_all_file = os.path.join('Thuc Nghiem', 'Selected Features', 'Results', '09.10', 'ALL_boruta_summary.csv')
all_results.to_csv(output_all_file, index=False)
logging.info(f"Kết quả tổng hợp Boruta cho tất cả các dataset được lưu vào file: {output_all_file}")
print(f"Kết quả tổng hợp Boruta cho tất cả các dataset được lưu vào file: {output_all_file}")


In [ ]:
## grid_search để tìm Best parameters: {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
import os
import logging
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

file_path = 'Gene Data/31552/data.trn.gz'
output_directory = os.path.join('Thuc Nghiem', 'Selected Features', 'Results', '04.10', '31552')
log_file = os.path.join(output_directory, 'experiment_log.log')
csv_file = os.path.join(output_directory, 'grid_search_results.csv')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.info('Bắt đầu thực hiện quá trình chọn lọc đặc trưng với Boruta.')

def load_data(file_path):
    try:
        df = pd.read_csv(file_path, header=None, sep='\s+')
        logging.info(f"Đọc dữ liệu từ file: {file_path}")
        return df
    except FileNotFoundError:
        logging.error(f"Không tìm thấy file: {file_path}")
        return None
def scale_data(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    logging.info("Dữ liệu đã được chuẩn hóa.")
    return X_scaled

df = load_data(file_path)
if df is not None:
    X = df.iloc[:, 1:]  # Đặc trưng
    y = df.iloc[:,-1]   # Nhãn

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train_scaled = scale_data(X_train)
    X_test_scaled = scale_data(X_test)

    svc = SVC(kernel='rbf', class_weight='balanced', random_state=42)
    param_grid = {
        'C': [1000, 10000, 100000, 1000000],
        'gamma': [0.01, 0.05, 0.005, 0.001, 0.0001],
        'kernel': ['rbf']
    }


    grid_search = GridSearchCV(svc, param_grid, cv=5, return_train_score=True)

    grid_search.fit(X_train_scaled, y_train)
    logging.info(f"Tham số tốt nhất: {grid_search.best_params_}")
    print("Best parameters:", grid_search.best_params_)

    results_df = pd.DataFrame(grid_search.cv_results_)
    results_df.to_csv(csv_file, index=False)
    logging.info(f"Kết quả GridSearchCV đã được ghi vào file: {csv_file}")

else:
    print("Không thể đọc dữ liệu.")




Best parameters: {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
